In [1]:
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')
import numpy as np
import sqlite3
import matplotlib
from datetime import date, timedelta
db_conn = sqlite3.connect('cases.db')
total_students = 85000
%load_ext autoreload
%autoreload 2
import techniques
pd.set_option('display.max_rows', None)

Loading BokehJS ...

In [2]:
df_summary = techniques.summarize(db_conn, total_students)
# smooth out the new cases reported on the 21st bc the lump reporting on that day messes with the forecasting
df_smooth = df_summary.copy()
cases = df_smooth.loc[pd.Timestamp(year=2021, month=9, day=21)]['New Student Cases']
cases_per_day = cases / 7.
for d in pd.date_range(start=pd.Timestamp(year=2021, month=9, day=15),
                       end=pd.Timestamp(year=2021, month=9, day=21),
                       freq='D'):
    df = df_smooth.append(pd.Series(name=d, dtype='float64'))
    df_smooth.loc[d, 'New Student Cases'] = cases_per_day
df_smooth = df_smooth.sort_index()

start_date = pd.Timestamp(year=2021, month=9, day=22)

model_high = techniques.Model2(
    df_smooth,
    start_date,
    total_students,
    seroprevalence = 0,
#    r0 = 1.62,
#    r0 = 1.42,
#    r0 = 1.0825,
    r0 = 1.001,
    quarantine_factor = 7.5,
    pct_long_covid = 0.135,
    pct_severe = 0.075,
    pct_death = 0.0001,
    quarantine_period = 10
)

model_low = techniques.Model2(
    df_smooth,
    start_date,
    total_students,
    seroprevalence = 0.25,
#    r0 = 1.62,
#    r0 = 1.42,
#    r0 = 1.0825,
    r0 = 1.001,
    quarantine_factor = 7.5,
    pct_long_covid = 0.135,
    pct_severe = 0.075,
    pct_death = 0.0001,
    quarantine_period = 10
)

for day in pd.date_range(start=start_date, end=pd.Timestamp(year=2021, month=12, day=23), freq='D'):
    model_high.tick()
    model_low.tick()

model_high.df

,Susceptible Students Remaining,% Susceptible Students Remaining,Rel pop dens,R_tick,New Cases,Active Cases,Total Cases,New Quarantined,Quarantined
2021-09-22,83609.000000,0.983635,1.000000,1.001000,55.000000,223.000000,279.000000,1335.000000,1335.000000
2021-09-23,83003.537500,0.976512,0.992758,0.993751,55.055000,255.755000,334.055000,412.912500,1918.162500
2021-09-24,82759.650934,0.973643,0.989841,0.990831,54.683642,284.863142,388.738642,410.127316,2136.473566
2021-09-25,82536.741486,0.971020,0.987175,0.988162,54.409399,310.786227,443.148041,408.070490,2330.896699
2021-09-26,82332.714018,0.968620,0.000000,0.000000,54.138469,333.846073,497.286510,406.038516,2503.845545
2021-09-27,82549.713965,0.971173,0.000000,0.000000,0.000000,300.461465,497.286510,0.000000,2253.460991
2021-09-28,82745.013917,0.973471,0.989666,0.990656,0.000000,270.415319,497.286510,0.000000,2028.114892
2021-09-29,82678.224459,0.972685,0.988868,0.989856,32.341255,275.715042,529.627765,242.559416,2067.862818
2021-09-30,82648.249239,0.972332,0.988509,0.989498,27.892000,276.035538,557.519765,209.189997,2070.266534
2021-10-01,82657.916535,0.972446,0.988625,0.989613,22.634107,271.066091,580.153872,169.755804,2032.995685


In [3]:
df = pd.DataFrame({
    'Actual Total Cases' : df_summary['Total Student Cases'],
    'Projected Total Cases (High)' : model_high.df['Total Cases'],
    'Projected Total Cases (Low)'  : model_low.df['Total Cases']
})
df.interpolate(method='time', limit_area='inside').plot()

Figure(id='1003', ...)

In [4]:
df = pd.DataFrame({
    'Actual Quarantine' : df_summary['Quarantined Students'],
    'Projected Quarantine (High)' : model_high.df['Quarantined'],
    'Projected Quarantine (Low)'  : model_low.df['Quarantined']
})
df.interpolate(method='time', limit_area='inside').plot()

Figure(id='1294', ...)